In [1]:
import numpy as np
import math
from random import random

In [2]:
# Library Gensim (Word2Vec examples)
### LOAD GOOGLE WORD2VEC... takes a few minutes
from gensim import models
# Load google's word2vec model
# languages
english = 'models/GoogleNews-vectors-negative300.bin.gz'
spanish = 'models/SBW-vectors-300-min5.bin.gz'
# load
model = models.Word2Vec.load_word2vec_format(english, binary=True)
model.init_sims(replace=True) # save memory
print 'model memory size, gb:', model.estimate_memory()['total']/(math.pow(1024,3))

model memory size, gb: 8.10250639915


Some helper functions that print and return an word arithmetic for:
- equation (classic similarity)
- equation (cosmul similarity)
- word (cosmul similarity)

In [9]:
# simple utility functions for printing
def print_similarities(similarity_list):
    for similarity in similarity_list:
        print similarity[0].ljust(18),similarity[1]
        
def similarities_p(pos=[],neg=[],topn=10,doPrint = True):
    if doPrint: 
        print "pos:",pos,"neg:",neg
    similarities = model.most_similar(pos,neg,topn)
    if doPrint: print_similarities(similarities)
    return similarities
    
def most_similar_cosmul_p(pos=[],neg=[],topn=10, doPrint = True):
    if doPrint: print "pos:",pos,"neg:",neg
    similarities = model.most_similar_cosmul(pos,neg,topn)
    if doPrint: print_similarities(similarities)    
    return similarities

def word_similar_cosmul_p(word,topn=10, doPrint = True):
    if doPrint: print "word:",word
    similarities = model.most_similar_cosmul([word],[],topn)
    if doPrint: print_similarities(similarities)    
    return similarities    

In [8]:
model['woman']

array([  9.15656984e-02,  -2.90509649e-02,  -3.87959108e-02,
        -4.04507108e-02,   4.44957837e-02,  -4.04507108e-02,
        -4.30248454e-02,   9.65301041e-03,   4.21055108e-02,
         1.82947535e-02,  -3.65895070e-02,  -1.29442275e-01,
        -2.37188265e-02,  -4.70699184e-02,  -1.02046113e-02,
         3.54863070e-02,  -7.06048757e-02,  -2.01334227e-02,
         2.35349592e-02,  -1.14916796e-02,  -1.09400786e-02,
        -1.81108862e-02,  -2.07769554e-02,  -1.53712705e-01,
         3.81523743e-03,  -8.75206292e-02,  -6.43534064e-02,
         9.92881060e-02,   1.31648675e-01,  -7.97982216e-02,
         5.40568605e-02,  -2.34430260e-03,  -8.49464908e-02,
        -3.97152416e-02,  -4.37603146e-02,   4.63344492e-02,
         1.15468390e-01,  -1.83866862e-02,  -3.60379070e-02,
         7.50176832e-02,  -5.92051297e-02,  -1.05723450e-02,
         5.95728643e-02,  -9.14737675e-03,   4.89085875e-02,
        -3.38315032e-02,  -2.86832321e-02,   1.33303478e-02,
        -1.15376459e-02,

In [12]:
most_similar_cosmul_p(['Spain','Berlin'],['Germany'],10)

pos: ['Spain', 'Berlin'] neg: ['Germany']
Madrid             0.899811029434
Paseo_del_Prado    0.811053216457
Barcelona          0.804468989372
Retiro_Park        0.799702107906
Buenos_Aires       0.796224355698
Bilbao             0.795319616795
Lisbon             0.794124364853
Bogota             0.793036460876
Gijón              0.792768776417
San_Sebastián      0.792483985424


[(u'Madrid', 0.8998110294342041),
 (u'Paseo_del_Prado', 0.8110532164573669),
 (u'Barcelona', 0.8044689893722534),
 (u'Retiro_Park', 0.7997021079063416),
 (u'Buenos_Aires', 0.7962243556976318),
 (u'Bilbao', 0.7953196167945862),
 (u'Lisbon', 0.7941243648529053),
 (u'Bogota', 0.7930364608764648),
 (u'Gij\xf3n', 0.7927687764167786),
 (u'San_Sebasti\xe1n', 0.7924839854240417)]

In [128]:
# DEMO

# 1 word arithmetic
#model.most_similar(positive=["woman",'girl']) # gunshot?
print "WORD ARITHMETIC"
model.most_similar(positive=["man"]) 
print model.most_similar(positive=["man",'boy']) 
# our helper fcts
similarities_and_p(['Germany'],['Spain'],topn=5)
most_similar_cosmul_p(['Germany'],['Spain'],topn=5)
# 2 similarity
print "\nSIMILARITY"
w1 = 'woman'
w2 = 'man'
print model.similarity(w1,w2)

# 4 non-matching
print "\nNON-MATCHING WORD"
print model.doesnt_match("breakfast cereal dinner lunch".split())

# 5 feature vector
vector = model['computer']
#print vector

# 6 similar by vector
vector[np.random.randint(0,300)] += random() * 0.5 - 0.05

print model.similar_by_vector(vector)

WORD ARITHMETIC
[(u'girl', 0.7885462045669556), (u'teenager', 0.7737530469894409), (u'woman', 0.7435684204101562), (u'teenage_girl', 0.7086503505706787), (u'teen_ager', 0.6568498611450195), (u'toddler', 0.6250425577163696), (u'youngster', 0.5938030481338501), (u'lad', 0.5809065699577332), (u'kid', 0.5788402557373047), (u'son', 0.577247142791748)]
pos: ['Germany'] neg: ['Spain']
Volker             0.458871692419
Bernd              0.453555166721
Wolfgang           0.444049060345
Jürgen             0.441551566124
Christoph          0.440208911896
pos: ['Germany'] neg: ['Spain']
Volker             1.45514786243
Bernd              1.40798211098
Wolfgang           1.39752185345
Dieter             1.39287662506
Jochen             1.39265882969

SIMILARITY
0.766401223441

NON-MATCHING WORD
cereal
[(u'computer', 1.115291953086853), (u'computers', 0.7021969556808472), (u'laptop_computer', 0.6121091842651367), (u'Computer', 0.5880930423736572), (u'laptop', 0.5856040120124817), (u'com_puter', 0.5

In [ ]:
# 'of' is not in the model

In [ ]:

#text1.concordance("monstrous")

In [29]:
from math import sqrt
b = model['o']
#print len(b)
#print b
print np.linalg.norm(b)


300
2.88349


In [26]:
# Library TextBlob
import textblob
# READ A TEXT
from textblob import TextBlob
book = open('../sci-fi-book.txt','r').read()
book_blob = TextBlob(book)

In [158]:
def tag(word):
    return TextBlob(word).tags[0]
#print tag('win')

('win', u'NN')


In [31]:
print "TAGS (grammar tree):"
blob_tags = book_blob.tags
print blob_tags

print 'NOUN PHRASES'
blob_noun_phrases = book_blob.noun_phrases
print blob_noun_phrases

TAGS (grammar tree):
[('The', u'DT'), ('Project', u'NNP'), ('Gutenberg', u'NNP'), ('EBook', u'NNP'), ('of', u'IN'), ('Valley', u'NNP'), ('of', u'IN'), ('the', u'DT'), ('Croen', u'NNP'), ('by', u'IN'), ('Lee', u'NNP'), ('Tarbell', u'NNP'), ('This', u'DT'), ('eBook', u'NN'), ('is', u'VBZ'), ('for', u'IN'), ('the', u'DT'), ('use', u'NN'), ('of', u'IN'), ('anyone', u'NN'), ('anywhere', u'RB'), ('at', u'IN'), ('no', u'DT'), ('cost', u'NN'), ('and', u'CC'), ('with', u'IN'), ('almost', u'RB'), ('no', u'DT'), ('restrictions', u'NNS'), ('whatsoever', u'RB'), ('You', u'PRP'), ('may', u'MD'), ('copy', u'VB'), ('it', u'PRP'), ('give', u'VB'), ('it', u'PRP'), ('away', u'RB'), ('or', u'CC'), ('re-use', u'VB'), ('it', u'PRP'), ('under', u'IN'), ('the', u'DT'), ('terms', u'NNS'), ('of', u'IN'), ('the', u'DT'), ('Project', u'NNP'), ('Gutenberg', u'NNP'), ('License', u'NNP'), ('included', u'VBD'), ('with', u'IN'), ('this', u'DT'), ('eBook', u'NN'), ('or', u'CC'), ('online', u'NN'), ('at', u'IN'), ('www.

In [77]:
print 'GET VERBS'

def get_tag(tags, query_tag):
    result = []
    for tag in tags:
        if tag[1] == query_tag:
            result.append(tag)
    return result

#print get_tag(blob_tags,'VBN')

#print book_blob.sentences

print "SENTENCE:"
sentence = book_blob.sentences[np.random.randint(0,len(book_blob.sentences))]
print sentence
print 'SENTIMENT'
print(sentence.sentiment)

GET VERBS
SENTENCE:
With you will
go my brother, Genner.
SENTIMENT
Sentiment(polarity=0.0, subjectivity=0.0)


In [87]:
print "SOME NLTK"
from nltk.corpus import treebank
import nltk

book_nltk_text = nltk.Text(book)
print book

print '\n',sentence
words = sentence.words
print words
# not working
#treebank.parsed_sents(sentence)
print sentence.pos_tags
nnps = get_tag(sentence.pos_tags,'NNP')

#for tree in parser.parse(words):
print nnps[0][0]
print book_nltk_text.concordance('yes')


SOME NLTK
The Project Gutenberg EBook of Valley of the Croen, by Lee Tarbell

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: Valley of the Croen

Author: Lee Tarbell

Release Date: July 8, 2008 [EBook #26010]

Language: English

Character set encoding: ASCII

*** START OF THIS PROJECT GUTENBERG EBOOK VALLEY OF THE CROEN ***




Produced by Greg Weeks, V. L. Simpson and the Online
Distributed Proofreading Team at http://www.pgdp.net





    [Transcriber's Notes:
        This etext was produced from Amazing Stories April 1949.
        Extensive research did not uncover any evidence that the
        U.S. copyright on this publication was renewed.

        Misspellings have been corrected.]



[Illustration]


VALLEY of the CROEN

By LEE TARBELL


    There was a mysterious golden 

In [64]:
word = 'angry'
print word,':'
print 'similars:'
p_similarities(model.most_similar([word],topn=20))
start_word = textblob.Word(word)
if len(start_word.synsets) != 0:
    print('synsets: ' + str(start_word.synsets))
    print('definitions: ' + str(start_word.definitions))
    synset_index = 0
    synset = start_word.synsets[synset_index]
    print('lemma_names',synset.lemmas())
    print('hypernyms:',synset.hypernyms())
    print('hyponyms:',synset.hyponyms())
    print('holonyms: ' + str(synset.member_holonyms()))
    print('meronyms: ' + str(synset.part_meronyms()))

angry :
similars:
irate              0.813892483711
enraged            0.770506680012
indignant          0.701343357563
incensed           0.688130795956
annoyed            0.68471622467
agitated           0.682100117207
frustrated         0.672057390213
furious            0.670225322247
unhappy            0.66462624073
outraged           0.659843683243
livid              0.657928168774
irritated          0.655557274818
exasperated        0.645732164383
angrier            0.634582161903
Angry              0.633797883987
infuriated         0.631404519081
impatient          0.627937555313
Protester_Thodoris 0.624717414379
disgusted          0.622331738472
angered            0.607131302357
synsets: [Synset('angry.a.01'), Synset('angry.s.02'), Synset('angry.s.03')]
definitions: [u'feeling or showing anger', u'(of the elements) as if showing violent anger', u'severely inflamed and painful']
('lemma_names', [Lemma('angry.a.01.angry')])
('hypernyms:', [])
('hyponyms:', [])
holonyms: []
merony

## DEMO
use nltk std chatbots to get an answer
then replace some words that by similar or with more positive sentiment
display old and new text with color showing the change. 
black: same
green: similar
red: higher sentiment

TODO: find words that indicate in which conceptual topic the sentence is

In [7]:
from nltk.chat import eliza, iesha, rude,suntsu
from textblob import TextBlob

#eliza.demo()
sentence = 'how is it that all birds are gay'
eli = eliza.eliza_chatbot
print 'eli',eli.respond(sentence)

print rude.rude_chatbot.respond(sentence)
print iesha.iesha_chatbot.respond(sentence)
print suntsu.suntsu_chatbot.respond(sentence)


eli Perhaps you can answer your own question.
I'm not even going to dignify that with an answer.
not tellin!! kekekekekeke ^_^
It is the rule in war, if our forces are ten to the enemy's one, to surround him; if five to one, to attack him; if twice as numerous, to divide our army into two.


In [10]:
# Take the suntsu response and change all words that have > .85 similarity
from IPython.core.display import display, HTML
#import progressbar
response = suntsu.suntsu_chatbot.respond('how is it that all birds are gay')

suntsu_response_blob = TextBlob(response)
print suntsu_response_blob
suntsu_words = suntsu_response_blob.words
print suntsu_response_blob.sentiment
reconfig = []
reconfig_how = []
i = 0 
#bar = progressbar.ProgressBar(max_value=len(suntsu_words),redirect_stdout=True)

for word in suntsu_response_blob.words:
    if word in model:
        similar = most_similar_cosmul_p(word,doPrint = False)
        suntsu_response_blob.words[i] = similar[0][0]    
        positiver = most_similar_cosmul_p([word,'king'],topn = 1,doPrint=False)[0][0]
        print word,similar[0], positiver
        #if tag(positiver) == tag(word):
        if model.similarity(positiver,word) > 0.7:
            reconfig.append(" ".join(positiver.split("_")))
            reconfig_how.append(1)
        elif similar[0][1] > 0.85:
            reconfig.append(" ".join(similar[0][0].split("_")))
            reconfig_how.append(2)
        else:
            reconfig.append(word)
            reconfig_how.append(0)
        print '>>>',reconfig[-1],reconfig_how[-1]
    i += 1
retune = TextBlob(" ".join(reconfig))
print retune
print retune.sentiment

i = 0 
retune_html = ''
span_temp = '<span style="color:z;">x </span>'
for w in reconfig:
    print suntsu_response_blob.words[i],w,reconfig_how[i]
    if reconfig_how[i] == 0:
        retune_html +=  span_temp.replace('z','black').replace('x',w)
    elif reconfig_how[i] == 1:
        retune_html +=  span_temp.replace('z','red').replace('x',w)
    else:
        retune_html +=  span_temp.replace('z','green').replace('x',w)
    #print retune_html
    i += 1

display(HTML('<h1>'+response+'</h1><div>Sentiment:'+str(suntsu_response_blob.sentiment)+'</div>'))
display(HTML('<h1>'+retune_html+'</h1><div>Sentiment:'+str(retune.sentiment)+'</div>'))

The good fighters of old first put themselves beyond the possibility of defeat, and then waited for an opportunity of defeating the enemy.
Sentiment(polarity=0.3499999999999999, subjectivity=0.37777777777777777)
The (u'This', 0.8512587547302246) kings
>>> This 2
good (u'great', 0.864574670791626) great
>>> great 1
fighters (u'fighter', 0.8711130023002625) kings
>>> fighter 2
old (u'yearold', 0.8658195734024048) boy
>>> yearold 2
first (u'second', 0.8985934257507324) second
>>> second 1
put (u'putting', 0.8948966264724731) putting
>>> putting 1
themselves (u'ourselves', 0.8430304527282715) kings
>>> themselves 0
beyond (u'beyong', 0.7696675658226013) kings
>>> beyond 0
the (u'this', 0.7968681454658508) kings
>>> the 0
possibility (u'possiblity', 0.9017102718353271) possiblity
>>> possiblity 1
defeat (u'defeats', 0.8950715661048889) kings
>>> defeats 2
then (u'when', 0.8256746530532837) kings
>>> then 0
waited (u'waited_patiently', 0.9026823043823242) waited_patiently
>>> waited patientl

In [196]:

#print len(wiki.words)   
#print wiki.words
#print wiki.tags
#noun_phrases = wiki.noun_phrases
#print noun_phrases
testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")
print testimonial.sentiment
testimonial.sentiment.polarity


Sentiment(polarity=0.39166666666666666, subjectivity=0.4357142857142857)


0.39166666666666666

In [117]:
from nltk import Text
#print book
book_Text = Text(book)
print book_Text
#book_Text.concordance("l")

<Text: T h e   P r o j...>


In [102]:
for np in noun_phrases:
    print np

project gutenberg ebook
croen
lee tarbell
restrictions whatsoever
project gutenberg license
title
croen author
lee tarbell release date
july
ebook
language
english character
ascii
start of this project gutenberg ebook valley of the croen
produced
greg weeks
v. l. simpson
online distributed proofreading team
//www.pgdp.net [
transcriber
notes
amazing stories april
extensive
u.s.
misspellings
] [
illustration
valley
croen
lee tarbell
golden statue
sudden death
illustration
tiny golden statue
huge counterpart
bad luck
black eyes
wide cicatrice
livid scar tissue
bad news
keele
mild voice
been
've run
korea
n't step
natural desire
small income
emerald mine
andes
indio
emerald mine
idea ...
too
especially
dirty rags
fusan
fusan
fingers work
korea
funny place
're immense forests
great mountains
think
superstitious
damp forests
scared
prospector 's meat
real emeralds
exquisite female figure
inset jewels
golden woman goddess
electric tingle
strong little soul intent
potent
queer stare
facial ex

In [62]:
model.most_similar(positive=['england', 'paris'], negative=['france'])
model.most_similar(positive=['hate',], negative=['love'])
model.most_similar(positive=['love',], negative=['hate'])

synset.lemmas()

#extension from love
#tell a story with code. 

[(u'enchanted', 0.36698639392852783),
 (u'adventuresome_spirit', 0.346050888299942),
 (u'pearl_diver', 0.3441880941390991),
 (u'adventurous_spirit', 0.3421488404273987),
 (u'childhood_sweetheart', 0.3396701216697693),
 (u'romance', 0.33354783058166504),
 (u'Luciana_Pedraza', 0.33328109979629517),
 (u'Clara_Wieck', 0.3316940665245056),
 (u'sultry_tango', 0.3311474323272705),
 (u'joys', 0.330089807510376)]